In [ ]:
import requests, zipfile, os
import pandas as pd
from datetime import datetime as dt, timedelta
from arcgis.features import GeoAccessor
from arcgis.gis import GIS

In [ ]:
gis = GIS('home')

In [ ]:
'''link to events zipfile from VIINA github'''
url = r'https://github.com/zhukovyuri/VIINA/raw/master/Data/events_latest.zip'
tempFile = r'/arcgis/home/events.zip'
REFRESH_RATE = 15 #minutes

In [ ]:
'''download the zip file to a temp dataset in the notebook files and open it as a pandas df'''
response = requests.get(url)
with open(tempFile,'wb') as outfile:
    outfile.write(response.content)
z = zipfile.ZipFile(tempFile)
df = pd.read_csv(z.open(z.filelist[0].filename))
os.remove(tempFile)

In [ ]:
def convertDTG(row):
    '''create ful DTG from date and time fields'''
    try:
        timeStr = '{} {}'.format(row['date'],row['time'])
        return dt.strptime(timeStr,'%Y%m%d %H:%M')
    except:
        return ''
    
def summarizeAggresor(row):
    '''summarize aggressor data into a single column'''
    if row['a_rus_b'] == 1:
        return 'Russia'
    if row['a_ukr_b'] == 1:
        return 'Ukraine'
    if row['a_civ_b'] == 1:
        return 'Civilians'
    if row['a_other_b'] == 1:
        return 'Other'
    else:
        return ''
    
def summarizeCasualties(row):
    '''summarize casualty data into a single column'''
    output = []
    if row['t_milcas_b'] == 1:
        output.append('Military')
    if row['t_civcas_b'] == 1:
        output.append('Civilian')
    return ','.join(output)

def summarizeMilitaryActions(row):
    '''summarize military activities to a single column'''
    output = []
    if row['t_aad_b'] == 1:
        output.append('AAA/MANPAD')
    if row['t_airstrike_b'] == 1:
        output.append('Airstrike')
    if row['t_armor_b'] == 1:
        output.append('Armor')
    if row['t_artillery_b'] == 1:
        output.append('Artillery')
    if row['t_firefight_b'] == 1:
        output.append('Firefight')
    if row['t_ied_b'] == 1:
        output.append('IED')
    return ','.join(output)

def remapControl(row):
    '''convert binary to True/False'''
    if row['t_control_b'] == 1:
        return 'True'
    else:
        'False'
def remapOccupation(row):
    '''convert binary to True/False'''
    if row['t_occupy_b'] == 1:
        return 'True'
    else:
        'False'
def remapProperty(row):
    '''convert binary to True/False'''
    if row['t_property_b'] == 1:
        return 'True'
    else:
        'False'
def remapCyber(row):
    '''convert binary to True/False'''
    if row['t_cyber_b'] == 1:
        return 'True'
    else:
        'False'
def remapMedical(row):
    '''convert binary to True/False'''
    if row['t_hospital_b'] == 1:
        return 'True'
    else:
        'False'

In [ ]:
'''apply remaps and summarizations to df'''
df['timestamp'] = df.apply(convertDTG,axis=1)
df = df[df['timestamp'] > dt.now()-timedelta(minutes=REFRESH_RATE)]
if len(df.index) > 0:
    df['aggressor'] = df.apply(summarizeAggresor,axis=1)
    df['casualties'] = df.apply(summarizeCasualties,axis=1)
    df['military_actions'] = df.apply(summarizeMilitaryActions,axis=1)
    df['control_changed'] = df.apply(remapControl,axis=1)
    df['occupation'] = df.apply(remapOccupation,axis=1)
    df['property_related'] = df.apply(remapProperty,axis=1)
    df['cyber_related'] = df.apply(remapCyber,axis=1)
    df['attack_on_medical_facility'] = df.apply(remapMedical,axis=1)
    
    to_drop = ['event_id','tempid','date','time_c','hours_c','hours','minutes','date_time','date_time_0','WID','TID','YRMO','MID','YRWK','YEAR']
    for c in list(df.columns):
        if c.endswith('_pred') or c.endswith('_b'):
            to_drop.append(c)
    df = df.drop(columns=to_drop)

In [ ]:
'''create a Spatially Enabled DataFrame using the arcgis.features.geoaccessor'''
sdf = pd.DataFrame.spatial.from_xy(df,'longitude','latitude')
sdf

In [ ]:
# Title: Ukraine_War_Events | Type: Feature Service 
updateLayer = gis.content.get("<PORTAL ITEM ID FOR EVENTS FEATURE SERVICE>").layers[0]
updateLayer

In [ ]:
'''add new events to feature service'''
if len(sdf.index) > 0:
    fs = sdf.spatial.to_featureset()
    updateLayer.edit_features(adds=fs)